# **Tutorial** - Topic Modeling with BERTopic
(last updated 08-06-2021)

In this tutorial we will be exploring how to use BERTopic to create topics from the well-known 20Newsgroups dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for. 


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic
!pip install flair

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [2]:
# IMPORTAR LISTA
from google.colab import drive
import dill as pickle
drive.mount('/content/drive/')
# open a file, where you stored the pickled data
file = open("/content/drive/MyDrive/Practica 3/essaysLemmatized.txt", 'rb')

# dump information to that file
essaysLemmatized = pickle.load(file)

# close the file
file.close()
processed=[]
for i in essaysLemmatized:
  string=""
  for x in i:
    string=string+x+" "
  processed.append(string)

Mounted at /content/drive/


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


hola?


```
from transformers import BertModel

# start from a pretrained bert-base-uncased model
model = BertModel.from_pretrained('bert-base-uncased')
```



In [4]:
#SOLO FUNCIONA ESTO
from bertopic import BERTopic
from flair.embeddings import TransformerDocumentEmbeddings
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(processed)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1874 [00:00<?, ?it/s]

2022-01-16 16:56:22,283 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-01-16 16:58:01,208 - BERTopic - Reduced dimensionality with UMAP
2022-01-16 16:58:21,383 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
roberta = TransformerDocumentEmbeddings('roberta-base')
topic_model = BERTopic(embedding_model=roberta, calculate_probabilities=True)

topics, probs = topic_model.fit_transform(processed)

In [ ]:
sentence_model  = SentenceTransformer('all-mpnet-base-v2')#import pre trained
topic_model = BERTopic(embedding_model=sentence_model,calculate_probabilities=True )

topics, probs = topic_model.fit_transform(processed)


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [47]:
freq = topic_model.get_topic_info(); freq.head(13)

,Topic,Count,Name
0,0,40649,0_love_good_friend_go
1,1,14452,1_friend_work_love_good
2,2,2230,2_darwing_darwe_crazy_todo
3,3,1508,3_love_bear_friend_good
4,4,334,4_bear_raise_friend_family
5,5,306,5_transplant_good_love_friend
6,6,180,6_hate_love_go_friend
7,-1,165,-1_love_good_friend_music
8,7,122,7_love_outgoing_friend_go


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [6]:
topic_model.get_topic(0)  # Select the most frequent topic

[('love', 0.02634813719097729),
 ('good', 0.022631551185589877),
 ('friend', 0.021114238460237626),
 ('go', 0.02080937323825513),
 ('thing', 0.02056831834024199),
 ('people', 0.02029773596404756),
 ('time', 0.020011771182474446),
 ('music', 0.019707709271804083),
 ('make', 0.019684922304286936),
 ('work', 0.018893091392391054)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

In [51]:
new_topics, new_probs = topic_model.reduce_topics(processed, topics, probs, nr_topics=9)


2022-01-16 17:11:10,901 - BERTopic - Reduced number of topics from 9 to 12


# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [34]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [35]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [36]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [37]:
topic_model.visualize_barchart(top_n_topics=11)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [40]:
topic_model.visualize_heatmap(n_clusters=8, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [41]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

In [42]:
import pandas as pd
topicVec = pd.DataFrame({'topic': topics, 'document': processed})


In [46]:
for t in range(0,8):
  x=topicVec[topicVec.topic == t]
  print(len(x))


40564
14423
2230
1508
334
306
180
122


## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(docs, topics, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('space', 0.011119596146117955),
 ('nasa', 0.0047697533973351915),
 ('shuttle', 0.0044533985251824495),
 ('orbit', 0.004129278694477752),
 ('spacecraft', 0.004011023125258004),
 ('satellite', 0.003783732360211832),
 ('moon', 0.003639954930862572),
 ('lunar', 0.0034753177228921146),
 ('the moon', 0.002821040122532999),
 ('mars', 0.0028033947303940923)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probs, nr_topics=60)

2021-10-17 06:05:02,666 - BERTopic - Reduced number of topics from 220 to 61


# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[1, 8, -1, 9, 12]

In [ ]:
topic_model.get_topic(5)

[('transplant', 0.042184290112045685),
 ('good', 0.02402329631021942),
 ('love', 0.02352261277506378),
 ('friend', 0.02233153281114789),
 ('work', 0.021554556420593302),
 ('music', 0.021371177377154875),
 ('go', 0.020326442618777137),
 ('make', 0.01973221620968864),
 ('thing', 0.01962196971017853),
 ('time', 0.018575686592463173)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
